## MNIST classification
### Goals
* Restoring the trained model with TensorFlow

### Contents
* How to load a model with a check point

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline  

In [2]:
# Build the same network model used for training

# Model parameters
# input
n_input_width = 28
n_input_height = 28
n_input_channel = 1

# 1st convolutional layer
n_conv1_patch_size = 3 
n_conv1_filter = 64

# 2nd convolutional layer
n_conv2_patch_size = 3 
n_conv2_filter = 64

# fully connected output layer
n_output  = 10 

# input and output placeholders
x = tf.placeholder(tf.float32, [None, n_input_width * n_input_height * n_input_channel])
y = tf.placeholder(tf.float32, [None, n_output])

# 1st convolutional layer variables
wc1 = tf.Variable(tf.random_normal([n_conv1_patch_size, n_conv1_patch_size, n_input_channel, n_conv1_filter], stddev=0.1))
bc1 = tf.Variable(tf.random_normal([n_conv1_filter], stddev=0.1))

# 2nd convolutional layer variables
wc2 = tf.Variable(tf.random_normal([n_conv2_patch_size, n_conv2_patch_size, n_conv1_filter, n_conv2_filter], stddev=0.1))
bc2 = tf.Variable(tf.random_normal([n_conv2_filter], stddev=0.1))

# fully connected output layer
# size reduced by four (2 pooling layers)
wf = tf.Variable(tf.random_normal([(n_input_width/4)*(n_input_height/4)*n_conv2_filter, n_output], stddev=0.1))
bf = tf.Variable(tf.random_normal([n_output], stddev=0.1))

# Reshape input: one hot vector to w x w x c tensor
input_r = tf.reshape(x, shape=[-1, n_input_width, n_input_width, n_input_channel])

# 1st Convolution layer: conv + bias + relu + pooling
# TensorFlow operators for CNNs
# https://www.tensorflow.org/api_guides/python/nn#Convolution
conv1 = tf.nn.conv2d(input_r, wc1, strides=[1, 1, 1, 1], padding='SAME')
bias1 = tf.nn.bias_add(conv1, bc1)
relu1 = tf.nn.relu(bias1)
pool1  = tf.nn.max_pool(relu1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 2nd Convolution layer
conv2 = tf.nn.conv2d(pool1, wc2, strides=[1, 1, 1, 1], padding='SAME')
bias2 = tf.nn.bias_add(conv2, bc2)
relu2 = tf.nn.relu(bias2)
pool2  = tf.nn.max_pool(relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Vectorize
dense = tf.reshape(pool2, [-1, wf.get_shape().as_list()[0]])

# Fully-connected layer
logits = tf.add(tf.matmul(dense, wf), bf)

# cost function and accuracy
corr = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

In [3]:
# Load the saved model
n_check = 10
path = "net/cnn_mnist.ckpt-" + str(n_check)

sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, path)

INFO:tensorflow:Restoring parameters from net/cnn_mnist.ckpt-10


In [4]:
# MNIST dataset
mnist = input_data.read_data_sets('data/', one_hot=True)
testImg    = mnist.test.images
testLabel  = mnist.test.labels

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [6]:
# Inference
batch_size = 100
num_batch = int(mnist.test.num_examples/batch_size)
accr_test_avg = 0.
for i in range(num_batch):
    batch_xs, batch_ys = mnist.test.next_batch(batch_size)
    accr_test_avg += sess.run(accr, feed_dict={x: batch_xs, y: batch_ys})/num_batch
print ("Test accuracy: %.3f" % (accr_test_avg))

Test accuracy: 0.990


In [13]:
# Let's measure the inference time
import time
t1 = time.time()
for i in range(num_batch):
    batch_xs, batch_ys = mnist.test.next_batch(batch_size)
    accr_test_avg += sess.run(accr, feed_dict={x: batch_xs, y: batch_ys})/num_batch
t2 = time.time()
print ('%0.3f ms for %d images' % ((t2-t1)*1000.0, mnist.test.num_examples))

335.920 ms for 10000 images
